In [1]:
!pip install ultralytics gradio opencv-python numpy gtts


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1


In [2]:
# ===================== INSTALL =====================
!pip install -q ultralytics gradio gtts opencv-python-headless

# ===================== IMPORTS =====================
import cv2
import numpy as np
import time
import uuid
from gtts import gTTS
from ultralytics import YOLO
import gradio as gr

# ===================== MODEL =====================
model = YOLO("yolov8n.pt")

# ===================== CONFIG =====================
CROWD_THRESHOLD = 60
FALL_FRAMES = 4
FIGHT_FRAMES = 6

MIN_DISTANCE = 150
MOTION_THRESHOLD = 18
IOU_THRESHOLD = 0.05

FRAME_SKIP = 5
VOICE_COOLDOWN = 6
last_voice_time = 0

# ===================== VOICE =====================
def generate_voice_safe(text):
    global last_voice_time
    now = time.time()
    if now - last_voice_time < VOICE_COOLDOWN:
        return None
    last_voice_time = now
    fname = f"alert_{uuid.uuid4().hex}.mp3"
    gTTS(text=text, lang="en").save(fname)
    return fname

# ===================== HELPERS =====================
def center(box):
    x1, y1, x2, y2 = box
    return ((x1+x2)//2, (y1+y2)//2)

def distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def detect_fall(box):
    x1, y1, x2, y2 = box
    return (x2-x1) > (y2-y1)

def iou(a, b):
    xA, yA = max(a[0], b[0]), max(a[1], b[1])
    xB, yB = min(a[2], b[2]), min(a[3], b[3])
    inter = max(0, xB-xA) * max(0, yB-yA)
    areaA = (a[2]-a[0])*(a[3]-a[1])
    areaB = (b[2]-b[0])*(b[3]-b[1])
    return inter / (areaA + areaB - inter + 1e-6)

# ===================== IMAGE =====================
def process_image(image):
    frame = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    res = model(frame)
    boxes = [b.cpu().numpy() for b in res[0].boxes.xyxy]
    count = len(boxes)

    alert = "✅ Normal"
    voice = "Area is normal."
    color = (0,255,0)

    if any(detect_fall(b) for b in boxes):
        alert = "🆘 FALL DETECTED"
        voice = "Emergency. A person has fallen."
        color = (0,0,255)

    elif count > CROWD_THRESHOLD:
        alert = "🚨 OVERCROWDING"
        voice = "Warning. Overcrowding detected."
        color = (0,165,255)

    voice_file = generate_voice_safe(voice)

    out = frame.copy()
    for b in boxes:
        x1,y1,x2,y2 = map(int,b)
        cv2.rectangle(out,(x1,y1),(x2,y2),color,3)

    out = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)
    return out, f"People Count: {count}", alert, voice_file

# ===================== VIDEO =====================
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    prev_centers = []
    fight_score = 0
    fall_score = 0
    frames = []
    voice_text = None
    fid = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        fid += 1
        if fid % FRAME_SKIP != 0:
            continue

        res = model(frame)
        boxes = [b.cpu().numpy() for b in res[0].boxes.xyxy]
        centers = [center(b) for b in boxes]

        alert = None
        color = (0,255,0)

        # FALL
        for b in boxes:
            if detect_fall(b):
                fall_score += 1
                if fall_score >= FALL_FRAMES:
                    alert = "🆘 FALL DETECTED"
                    voice_text = "Emergency. A person has fallen."
                    color = (0,0,255)
            else:
                fall_score = 0

        # FIGHT
        if len(centers)>=2 and len(prev_centers)==len(centers):
            for i in range(len(centers)):
                for j in range(i+1,len(centers)):
                    d = distance(centers[i], centers[j])
                    m = distance(prev_centers[i], centers[i])
                    ov = iou(boxes[i], boxes[j])
                    if d < MIN_DISTANCE and (m > MOTION_THRESHOLD or ov > IOU_THRESHOLD):
                        fight_score += 1
                    else:
                        fight_score = max(0, fight_score-1)

        if fight_score >= FIGHT_FRAMES:
            alert = "⚠️ FIGHT DETECTED"
            voice_text = "Critical alert. Violent activity detected."
            color = (0,0,255)

        # CROWD
        if alert is None and len(boxes) > CROWD_THRESHOLD:
            alert = "🚨 OVERCROWDING"
            voice_text = "Warning. Overcrowding detected."
            color = (0,165,255)

        out = frame.copy()
        for b in boxes:
            x1,y1,x2,y2 = map(int,b)
            cv2.rectangle(out,(x1,y1),(x2,y2),color,3)

        if alert:
            cv2.putText(out,alert,(30,60),
                        cv2.FONT_HERSHEY_SIMPLEX,1.2,(0,0,255),3)

        prev_centers = centers.copy()
        frames.append(cv2.cvtColor(out, cv2.COLOR_BGR2RGB))

    cap.release()
    voice_file = generate_voice_safe(voice_text) if voice_text else None
    return frames, voice_file

# ===================== UI =====================
css = """
body {background:#020617;color:#e5e7eb;}
#title{text-align:center;font-size:48px;color:#38bdf8;font-weight:900}
#sub{text-align:center;color:#7dd3fc;margin-bottom:25px}
.panel{background:#020617;padding:20px;border-radius:14px;border:1px solid #1e293b}
.gradio-container{max-width:1200px}
"""

with gr.Blocks(css=css,title="SENTINEL") as demo:
    gr.HTML("<div id='title'>🛡️ SENTINEL</div><div id='sub'>Smart Crowd Monitoring & Alert System</div>")

    with gr.Tab("📸 Image Monitoring"):
        with gr.Row():
            gr.Column(scale=1)
            with gr.Column(scale=3):
                gr.HTML("<div class='panel'>")
                img_in = gr.Image(type="numpy")
                img_out = gr.Image()
                cnt = gr.Textbox(label="People Count")
                alert = gr.Textbox(label="Alert")
                aud = gr.Audio(autoplay=True)
                gr.Button("Analyze Image").click(process_image,img_in,[img_out,cnt,alert,aud])
                gr.HTML("</div>")
            gr.Column(scale=1)

    with gr.Tab("🎥 Video Monitoring"):
        with gr.Row():
            gr.Column(scale=1)
            with gr.Column(scale=3):
                gr.HTML("<div class='panel'>")
                vid = gr.Video()
                gal = gr.Gallery()
                aud2 = gr.Audio(autoplay=True)
                gr.Button("Analyze Video").click(process_video,vid,[gal,aud2])
                gr.HTML("</div>")
            gr.Column(scale=1)

demo.launch(share=True)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/tmp/ipython-input-2635060494.py:176: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css,title="SENTINEL") as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2614ac19edbecb77fe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
